<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/210728_210810/NLPwithStanford.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 스탠포드 자연어처리 강의
<https://blog.naver.com/jujbob/221147988848>
---
# 1
## 자연어처리(Natural Language Processing)
컴퓨터가 인간의 말을 이해할 수 있도록 처리하는 것   

## Deep Learning with NLP?
딥러닝은 기계학습의 소분류   
- 기존의 자연어 처리
  - 문제를 인간의 관점에서 해석된 문제의 정의를 인간이 직접 'feature'을 선택해서 기계학습
  - linear 모델이 주류로 활용
  - input과 features를 인간이 정하고 학습하면 기계는 weight를 찾는 산술적 optimization 역할이 됨

- 딥러닝
  - 기계가 raw-data로 부터 자동으로 문제의 feature을 추출하고 최적의 판단(산술 연산)

## 자연어처리가 왜 어렵지
- 글자(말)의 모호성: 복잡한 단어 조합
- 의미의 모호성: 상환, 상식에 따라 달라지는 의미

## 세부분야
- 감정분석 sentiment analysis
- 기계번역 machine translation
- 음성인식 voice recognition
- 질의응답 question answering
- 구문분석 parsing
---
# 2
## NLP에서 단어를 처리하는 방법
- one-hot encoding
  - 단어를 문자(symbol) 그 자체로 봐서 컴퓨터가 각 단어를 식별하기 위해 각 단어에 유일한 벡터 부여
  - 단어를 symbolic하게 표현하니까 단어 고유의 개념, 의미 표현 불가능
  - 확률적 언어모델 probablistic language model
- distributional similarity based representation
  - 비슷한 의미를 가지는 단어를 벡터상 가까운 위치에 두는 방법
  - 어떤 한 단어를 표현할 때 주변의 단어들과 해당 단어의 분포를 이용해 단어의 vector 표현하는 것으로 구현
  - 단어들의 수치화된 분포 표현은 단어들의 유사도에 기반하여 표현
  - __유사도(similarity)__는 해당 단어와 함께 출몰한 주변단어를 기준으로 결정되기 때문에 단어가 가지는 문법적, 개념적 의미 표현 가능

## 단어의 위치 분포적 표현(word embeddings)
- TF-TDF, SVD 등을 사용해서도 분포적 표현은 가능, skip-gram, CBOW(word2vec)가 나온 이후 의미를 내포한 단어표현(distributional representation) == word embedding 으로 생각
- word embedding
  - 임의의 단어 w가 주어졌을 때 주변 단어들을 예측하는 확률 모델
    - 'dog' 주변에 어떤 단어들이 자주 나올까?
    - 주변 단어들을 예측하며 parameter 값 조절

## Skip-gram
#### 개념, loss 구하기
- word embedding을 학습하는 방법 중 하나
- center word 기준으로 양옆 m=2개의 단어가 어떤 단어가 나올 때 최적의 확률 값을 가지는지를 예측하는 모델
  - arg max[P(Wt-2|Wt)*P(Wt-1|Wt)*P(Wt+1|Wt)*P(Wt+2|Wt)] / t=1~T까지
  - -1/T가 추가되고 log 씌움 : negative log-likelihood 방식으로 loss 계산
  - negative log-likelihood는 기계학습에서 많이 사용되는데, 확률수식이 최대확률값을 반환하는 parameters을 찾는 문제였다면, 기계학습에서는 손실값을 최소로하는 parameters를 찾는 일로 말만 바뀐 것
  - log를 취하면 log 내 1이하로 매우 작은 P확률값을 크게 하고 계산이 편해지고  loss 분포를 reqular하는 효과
#### 구현
1. 문서 D에서 사용된 단어Wt를 모두 추출하여 one-hot encoding W 생성
2. 최종적으로 우리가 학습할 word embedding parameter V(center word vector)을 랜덤 초기화
3. scoring을 위해 parameter U(context word vector) 랜덤 초기화
4. forward 연산: Wt*V = vc(word embedding) vc*U = uc(embedding*scoring parm) softmax(uc)= y'로 확률분포 구함
5. loss 구함: cross-entropy 사용해서 확률분포와 실제 정답 간 확률간 거리 비교
6. backword 연산: gradient descent
7. parameter update: gradient descent 결과에 따라 parameter V, U 업데이트

---
# 3
## skip-gram with Negative sampling
- skip-gram에서 아는 단어가 n개면 n번 벡터연산한다. 벡터연산이 너무 많다. 단어를 줄여보자
- nagative sampling
  - 단어를 정해진 규칙 안에서 랜덤으로 선택하는데, 고의적으로 여러 개의 오답이 될만한 후보를 랜덤하게 선택해서 확률 값에 nagative취하는 방법
  - 정답이 될 수 있는 후보쌍의 확률값을 최대로 만들고, 오답이 될 수 있는 후보쌍의 확률을 최소로 한다.

## GloVe model
- skip-gram은 주변 단어로 인식하는 범위인 local context window(1~20)을 지정함. window size를 무한정 늘려 문서 전체로 한다면 각 단어의 벡터는 서로 변별력을 찾기 어렵다. 
  - skip-gram은 window size를 늘리면 전체 단어의 동반 출현 빈도수와 같은 통계 정보(global co-occurrence statistics)를 내포하지 못함
- global co-occurrence statistics 정보를 구하는 다른 방식: count-based matrix를 만드는 방법
  - 주로 matrix 만들고 SVD, LSA, HAL, LSI등으로 차원 축소 시킴
  - 전체적인 통계 정보를 내포하는데 유리, skip-gram은 좀 더 작은 범위인 단어간의 유사도를 파악하는데 서로 장점이 있다.
- GloVe model 
  - global co-occurrence statistics 정보를 포함한 direct prediction word embedding, 
  - 전체적인 통계정보 + skip-gram과 같은 direct prediction 방식의 장점을 모두 가짐

1. 전체 학습데이터로부터 co-occuence matrix 'X' 생성
2. 손실함수 j 정의(least square objective)
3. forward, backward, gradient discent 업데이트
  - Xij: center word i에 대한 context word j의 출현 빈도(스칼라)
  - Xi: 전체 학습데이터에 대한 center word i의 출현 빈도(스칼라)

#### 학습 전 복습
- skip-gram: 주변어 예측 확률 = P(context|center), 우리는 이미 co-occurrence matrix X가 있기 때문에 loss function에 별도의 input을 다시 넣지 않고 활용해서 학습하자
- 각 center word마다 context word를 한 번씩 다 돌리면서 확률 구함, 연산속도 느림

#### 학습
- GloVe에서 우리는 cross-entropy 대신 (weighted)least square objective라는 새로운 손실함수 사용(mean squared error와 비슷)
- 중심어에 대한 j번째 context 단어가 많이 출현했다면 exp(UjVi)값이 커야 loss 0 에 가깝다.
  - normalization을 위해 log 추가

## word embedding evaluation
- intrinsic(내적 평가): word embedding만을 이용해 내부 단어간의 유사도를 통해 평가
  - human judgement 평가: 사람이 직접 평가 데이터를 생성. 주로 두 단어의 유사도를 직접 점수매김(tiger;cat = 10점 중 8점)
  - semantic(의미), syntactic(문법) 평가: word2vec논문에서 소개된 dataset을 기준으로 cosine similarity를 평가(여러 capital-contry 쌍의 결과들이 얼마나 비슷한가(서울-한국), (도쿄-일본)의 결과벡터유사도)
- extrinsic(외적 평가): 실제 문제에 직접 적용하여 성능 평가(개체명인식기(named entity recognition) 각종 자연어처리 시스템에 embedding을 직접 사용하여 시스템의 성능을 측정)

- 내적평가에서 유사도 평가: word embedding은 결국 vector여서 vector간 유사도라 하면 코사인 유사도나 벡터간의 거리 계산하는 Euclidean distance 사용
  - ex.수도-국가 예제처럼  man:woman :: king:d? 에서 최대값을 가지는 후보d를 찾아 정답 queen을 찾는 형식으로 진행, king - man + woman = queen

## word embedding은 어떤 의마와 문법 정보를 내포할까?
- king 주변에는 royal, familiy, emperor 등이 예상되고 이런 단어들은 queen의 주변어로도 자주 등장하지만 king/ queen을 구분할 수 있는 주변단어가 몇가지 있을 수 있다.    
- "the king was a man", "the queen was a woman", "he will be a man", "he will be a king", "he will be a uncle"   
- 중심어 king은 man, he, uncle과 문법적, 의미적으로 she, woman보다 상대적으로 많이 등장할 것이고 문장에서 king의 자리는 저 단어들로 대체될 수 있다. ("she is the wife of the king"같은 문장이 나올 수 있어서 상대적으로 생각하자)   
- 결과적으로, word embedding은 의미적, 문법적 정보를 단어의 동반출현 빈도를 통해 상대적으로 학습


